In [1]:
from pathlib import Path
import collections
import fiona

from src.graph.graph_generator import GraphGenerator
from src.graph.path_schema import PairsList

# Wczytywanie grafów
- grafu reprezentującego ściezki rowerowe, które zostały połączone z atrakcjami
- grafu reprezentującego drogi, chodniki itp., które zostały połączone z atrakcjami

In [2]:
graph_generator = GraphGenerator()

roads_graph = graph_generator.read_graph("data/roads_w_attractions.pickle")
bike_paths_graph = graph_generator.read_graph("data/bikepaths_w_attractions.pickle")

# Wczytywanie plików z zapisanymi trasami pomiędzy atrakcjami

In [3]:
PATHS_DIR = "out"

paths_dir = Path(PATHS_DIR)

paths_files = list(paths_dir.glob("data*.json"))

print(len(paths_files))

40


In [4]:
files_contents = PairsList(pairs=[])

for file in paths_files:
    content = PairsList.parse_file(file)
    files_contents.pairs = [*files_contents.pairs, *content.pairs]

In [5]:
len(files_contents.pairs)

40000

In [6]:
for pair in files_contents.pairs[:20]:
    len1 = pair.other_path.length if pair.other_path else float("inf")
    len2 = pair.bike_path.length if pair.bike_path else float("inf")
    len3 = pair.safer_bike_path.length if pair.safer_bike_path else float("inf")
    start_title = pair.start.name[:20] if pair.start.name else pair.start.id
    end_title = pair.end.name[:20] if pair.end.name else pair.end.id
    print(f"Z: {start_title:>20} do: {end_title:>20}", end=" -  ")
    print(f"Roads: {len1:7.3f} km,\tBike Lanes: {len2:7.3f} km,\tSafer Bike Lanes:{len3:7.3f} km")

Z:           Rzeźba lwa do:               Ślepak -  Roads:   5.060 km,	Bike Lanes:   5.578 km,	Safer Bike Lanes:  5.691 km
Z:               Piątka do:            Bankierek -  Roads:   2.688 km,	Bike Lanes:   3.172 km,	Safer Bike Lanes:  3.172 km
Z:                 Sowy do:               Trener -  Roads:   2.276 km,	Bike Lanes:   2.373 km,	Safer Bike Lanes:  2.373 km
Z:          Rozkwietnik do:                  486 -  Roads:   2.035 km,	Bike Lanes:   2.418 km,	Safer Bike Lanes:  2.425 km
Z:            Esencjusz do:                  390 -  Roads:   1.552 km,	Bike Lanes:   1.611 km,	Safer Bike Lanes:    inf km
Z:                Zebra do:                  511 -  Roads:   2.206 km,	Bike Lanes:   2.198 km,	Safer Bike Lanes:  2.198 km
Z:     Ma(STE)rche(FEK) do:      Galeria Miejska -  Roads:   1.534 km,	Bike Lanes:   1.393 km,	Safer Bike Lanes:    inf km
Z: Mauzoleum Piastów Śl do:                  548 -  Roads:   1.554 km,	Bike Lanes:   1.604 km,	Safer Bike Lanes:  2.613 km
Z:          Anio

# Sprawdźmy jaki był stosunek długości dróg

In [7]:
def get_ratios(p1, p2, p3):
    len1 = p1.length if p1 else None
    len2 = p2.length if p2 else None
    len3 = p3.length if p3 else None
    ratio1 = 0.0
    ratio2 = 0.0
    if len1 is not None and len2 is not None:
        ratio1 = len2 / len1
    if len1 is not None and len3 is not None:
        ratio2 = len3 / len1
    return ratio1, ratio2

ratios = [(i, *get_ratios(pair.other_path, pair.bike_path, pair.safer_bike_path)) for i, pair in enumerate(files_contents.pairs)]
ratios_filtered = list(filter(lambda x: x[1] != 0 or x[2] != 0, ratios))

In [8]:
print(f"Ratios len: {len(ratios)}, filtered len: {len(ratios_filtered)}")

Ratios len: 40000, filtered len: 37469


In [9]:
ratios_filtered[:20]

[(0, 1.1024852186022238, 1.1248009710622415),
 (1, 1.1800221651768918, 1.1800221651768918),
 (2, 1.0424940379846415, 1.0424940379846415),
 (3, 1.1880991137154318, 1.1918221911263427),
 (4, 1.0382575510395458, 0.0),
 (5, 0.9963989257541389, 0.9963989257541389),
 (6, 0.9077646675366177, 0.0),
 (7, 1.0323687911113106, 1.6817584321836105),
 (8, 1.067017385075157, 1.2312501153783992),
 (10, 1.094314130597677, 1.123504806032508),
 (11, 0.9622186190246654, 1.1426957006753453),
 (12, 1.0936916872652238, 1.0936916872652238),
 (13, 1.3223563923431072, 0.0),
 (14, 1.0904084452281764, 1.1617651530436575),
 (15, 1.0586803408479482, 1.136521054999785),
 (16, 1.1406716425378625, 1.1695623631423693),
 (17, 1.1754371434022433, 0.0),
 (19, 1.0218027290082823, 1.0916882958285883),
 (20, 1.1676948403021745, 0.0),
 (21, 1.0231827045320054, 0.0)]

# Algorytm znajdowania braków w infrastrukturze rowerowej
Spróbujmy jakoś uszeregować ścieżki znalezione pomiędzy atrakcjami ze względu na dostępność dla rowerów

In [10]:
print(max(ratios_filtered, key=lambda x: x[1]))
print(max(ratios_filtered, key=lambda x: x[2]))

(24839, 2.838886526001886, 0.0)
(5065, 1.015611407209453, 4.036488944296039)


In [11]:
max_ratio1 = max(ratios_filtered, key=lambda x: x[1])[1]
max_ratio2 = max(ratios_filtered, key=lambda x: x[2])[2]
points = {}
points1 = {}

for pair_index, ratio1, ratio2 in ratios_filtered:
    pair =files_contents.pairs[pair_index]
    if pair.other_path is not None and pair.bike_path is not None:
        ratio = ratio1 if ratio1 != 0 else max_ratio1
        for (i, j) in zip(pair.other_path.edges, pair.other_path.edges[1:]):
            d_points = ratio - 1.0 if ratio >= 1.0 else 0.0
            points[(i, j)] = points.get((i, j), 0) + d_points

    if pair.bike_path is not None and pair.safer_bike_path is not None:
        ratio = ratio2 if ratio2 != 0 else max_ratio2
        for (i, j) in zip(pair.bike_path.edges, pair.bike_path.edges[1:]):
            d_points = ratio - 1.0 if ratio >= 1.0 else 0.0
            points1[(i, j)] = points.get((i, j), 0) + ratio - 1.0

In [12]:
max(points.items(), key=lambda x: x[1])

((1077, 1078), 343.9107216083429)

In [13]:
max(points1.items(), key=lambda x: x[1])

((2385, 2384), 238.67929585579787)

In [47]:
missing_bikelanes = {}
for k, v in points.items():
    if missing_bikelanes.get(v) is None:
        missing_bikelanes[v] = []
    missing_bikelanes[v].append(k)

missing_safer_bikelanes = {}
for k, v in points1.items():
    if missing_safer_bikelanes.get(v) is None:
        missing_safer_bikelanes[v] = []
    missing_safer_bikelanes[v].append(k)

In [48]:
missing_bikelanes = dict(filter(lambda x: x[0] >= 100, missing_bikelanes.items()))
missing_safer_bikelanes = dict(filter(lambda x: x[0] >= 100, missing_safer_bikelanes.items()))

In [49]:
roads_edges_ids = [i[1] for i in missing_bikelanes.items()]
bikepaths_edges_ids = [i[1] for i in missing_safer_bikelanes.items()]

In [50]:
paths_nodes = []

for path in roads_edges_ids:
    path_nodes = []
    for i, j in path:
        if len(path_nodes) == 0 or path_nodes[-1] != i:
            path_nodes.append(i)
        if path_nodes[-1] != j:
            path_nodes.append(j)
    paths_nodes.append(path_nodes)

proposed_bikelanes_coords = [[roads_graph.nodes[node]["pos"] for node in path] for path in paths_nodes]

In [51]:
paths_nodes = []

for path in bikepaths_edges_ids:
    path_nodes = []
    for i, j in path:
        if len(path_nodes) == 0 or path_nodes[-1] != i:
            path_nodes.append(i)
        if path_nodes[-1] != j:
            path_nodes.append(j)
    paths_nodes.append(path_nodes)

proposed_safer_bikelanes_coords = [[bike_paths_graph.nodes[node]["pos"] for node in path] for path in paths_nodes]

In [52]:
with fiona.open('data/lines.shp') as src:
    meta = src.meta

In [53]:
with fiona.open("out/propose_bike_paths.shp", "w", **meta) as dst:
    for i, path in enumerate(proposed_bikelanes_coords):
        path = {
            "type": "Feature",
            "id": i + 1,
            "properties": collections.OrderedDict([('osm_id', f'{i + 1000000}'),
               ('name', None),
               ('highway', 'motorway'),
               ('waterway', None),
               ('aerialway', None),
               ('barrier', None),
               ('man_made', None),
               ('z_order', 9),
               ('other_tags',
                '')]),
            "geometry": {
                "type": "LineString",
                "coordinates": path
            }
        }
        dst.write(path)

In [54]:
with fiona.open("out/propose_safer_bike_paths.shp", "w", **meta) as dst:
    for i, path in enumerate(proposed_safer_bikelanes_coords):
        path = {
            "type": "Feature",
            "id": i + 1,
            "properties": collections.OrderedDict([('osm_id', f'{i + 2000000}'),
               ('name', None),
               ('highway', 'motorway'),
               ('waterway', None),
               ('aerialway', None),
               ('barrier', None),
               ('man_made', None),
               ('z_order', 9),
               ('other_tags',
                '')]),
            "geometry": {
                "type": "LineString",
                "coordinates": path
            }
        }
        dst.write(path)